# Progetto finale
# Antonio Lombardo - Matricola 878179

You have to work on the University dataset. It contains three different university rankings:

    The Times Higher Education World University Ranking, shortly Times,
    the Academic Ranking of World Universities, shortly Shanghai,
    the Center for World University Rankings, shortly cwur.

##### Notes

    It is mandatory to use GitHub for developing the project.
    The project must be a jupyter notebook.
    There is no restriction on the libraries that can be used, nor on the Python version.
    All questions on the project must be asked in a public channel on Zulip.

Importo le librerie utilizzate.

In [1]:
import pandas as pd
import numpy as np
import re

Effettuo un preprocessing dei dati per verificare se nei database da utilizzare siano presenti righe contenenti valori nulli o valori anomali.

### Preprocessing database timesDb

In [2]:
timesDb = pd.read_csv('dataset_progetto/timesData.csv')
timesDb.head(3)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011


Analizzo il tipo di ogni colonna del database timesDb.

In [3]:
timesDb.dtypes

world_rank                 object
university_name            object
country                    object
teaching                  float64
international              object
research                  float64
citations                 float64
income                     object
total_score                object
num_students               object
student_staff_ratio       float64
international_students     object
female_male_ratio          object
year                        int64
dtype: object

Effettuo un controllo solamente sulla colonna 'university_name' dato che un valore nullo non avrebbe senso. A differenza, per le altre colonne, è possibile che il dato non sia stato rilevato e quindi accetto valori nulli se sono presenti. 

In [4]:
timesDb['university_name'].isnull().any()

False

Non sono presenti missing value per i valori della colonna 'university_name'.

### Preprocessing database shanghaiDb

In [5]:
shanghaiDb = pd.read_csv('dataset_progetto/shanghaiData.csv')
shanghaiDb.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005


Analizzo il tipo di ogni colonna del database shanghaiDb.

In [6]:
shanghaiDb.dtypes

world_rank          object
university_name     object
national_rank       object
total_score        float64
alumni             float64
award              float64
hici               float64
ns                 float64
pub                float64
pcp                float64
year                 int64
dtype: object

In [7]:
shanghaiDb['university_name'].isnull().any()

True

In [8]:
shanghaiDb[shanghaiDb['university_name'].isnull() == True]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
3896,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013


Decido di rimuovere la riga contente valori nulli per university_name

In [9]:
shanghaiDb.drop([3896], inplace = True)

In [10]:
shanghaiDb['university_name'].isnull().any()

False

Adesso non sono presenti missing value per i valori della colonna 'university_name'.

### Preprocessing database cdwurDb

In [11]:
cwurDb = pd.read_csv('dataset_progetto/cwurData.csv')
cwurDb.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


Analizzo il tipo di ogni colonna del database cwurDb.

In [12]:
cwurDb.dtypes

world_rank                int64
institution              object
country                  object
national_rank             int64
quality_of_education      int64
alumni_employment         int64
quality_of_faculty        int64
publications              int64
influence                 int64
citations                 int64
broad_impact            float64
patents                   int64
score                   float64
year                      int64
dtype: object

In [13]:
cwurDb['institution'].isnull().any()

False

Non sono presenti missing value per i valori della colonna institution.

# 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes.

Per restituire l'anno più recente, o meno recente, utilizzo il metodo idxmax(), o idxmin(), che ritorna l'indice esplicito.

In [14]:
timesDb_most_recent = timesDb.loc[timesDb.groupby('university_name')['year'].idxmax()]
timesDb_most_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


In [15]:
timesDb_least_recent = timesDb.loc[timesDb.groupby('university_name')['year'].idxmin()]
timesDb_least_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012


# 2. For each university, compute the improvement in income between the least recent and the most recent data points.

Per calcolare il miglioramento del reddito fra il dato più recente e meno recente, creo delle nuove tabelle partendo da quelle create al punto 1 e le unisco aggiungendo il suffisso _max e _min per la colonna 'income'. 

In [16]:
timesDb_least_recent_income = timesDb_least_recent[['university_name', 'income']]
timesDb_most_recent_income = timesDb_most_recent[['university_name', 'income']]

In [17]:
times_db_union = pd.merge(timesDb_least_recent_income, timesDb_most_recent_income, on = ['university_name'], 
                          suffixes = ('_min', '_max'))
times_db_union.head()

,university_name,income_min,income_max
0,AGH University of Science and Technology,-,-
1,Aalborg University,36.4,43.7
2,Aalto University,61.9,61.6
3,Aarhus University,61.5,68.3
4,Aberystwyth University,35.5,31.3


Analizzando i valori delle colonne 'income_max' e 'income_min' si nota la presenza di alcuni valori non rilevati indicati con '-'.

Si decide, quindi, di rimuovere tali valori.

Effettuo un controllo sulla colonna 'income_max'.

In [18]:
times_db_union[times_db_union['income_max'].str.contains('-')]['income_max'].count()

44

In [19]:
times_db_union_clear = times_db_union.drop(times_db_union[times_db_union['income_max'].str.contains('-')].index)

In [20]:
times_db_union_clear[times_db_union_clear['income_max'].str.contains('-')]['income_max'].count()

0

Effettuo un controllo sulla colonna 'income_min'.

In [21]:
times_db_union_clear[times_db_union_clear['income_min'].str.contains('-')]['income_min'].count()

70

In [22]:
times_db_union_clear.drop(times_db_union_clear[times_db_union['income_min'].str.contains('-')].index, inplace= True)

<ipython-input-22-3c027c266bb8>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  times_db_union_clear.drop(times_db_union_clear[times_db_union['income_min'].str.contains('-')].index, inplace= True)


In [23]:
times_db_union_clear[times_db_union_clear['income_min'].str.contains('-')]['income_min'].count()

0

In [24]:
times_db_union_clear.head()

,university_name,income_min,income_max
1,Aalborg University,36.4,43.7
2,Aalto University,61.9,61.6
3,Aarhus University,61.5,68.3
4,Aberystwyth University,35.5,31.3
5,Adam Mickiewicz University,28.7,28.7


Effettuo una conversione per le colonne 'income_min' e 'income_max' che da object diventano float per il calcolo della differenza del reddito.

In [25]:
times_db_union_clear['improvement'] = times_db_union_clear['income_max'].astype(float) \
                                        - times_db_union_clear['income_min'].astype(float)
times_db_union_clear

,university_name,income_min,income_max,improvement
1,Aalborg University,36.4,43.7,7.3
2,Aalto University,61.9,61.6,-0.3
3,Aarhus University,61.5,68.3,6.8
4,Aberystwyth University,35.5,31.3,-4.2
5,Adam Mickiewicz University,28.7,28.7,0.0
...,...,...,...,...
812,Zhejiang University,70.3,96.2,25.9
813,École Normale Supérieure,30.7,37.1,6.4
814,École Normale Supérieure de Lyon,26.1,31.7,5.6
816,École Polytechnique Fédérale de Lausanne,38.0,65.4,27.4


# 3. Find the university with the largest increase computed in the previous point.

Tenendo in considerazione la richiesta del testo, non ci è assicurato che solamente una sola università abbia avuto il più alto improvement. Per tale motivo è bene usare il metodo max(), invece di idxmax() che ritorna solamente la prima riga che soddisfa tale condizione, in modo da avere tutte le università con pari improvement più alto.

In [26]:
times_db_union_clear.loc[times_db_union_clear['improvement'].idxmax()]

university_name    TU Dresden
income_min               31.9
income_max               99.7
improvement              67.8
Name: 428, dtype: object

In [27]:
times_db_union_clear[times_db_union_clear['improvement'] == times_db_union_clear['improvement'].max()]

,university_name,income_min,income_max,improvement
428,TU Dresden,31.9,99.7,67.8


# 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range.

Dall'analisi del database è emerso che nella colonna 'world_rank' possono presentarsi 3 diverse situazioni:

    ·valore numerico intero, esempio 54, 31, etc...
    ·valore numeri intero preceduto da =, esempio =106, =106, etc...
    ·intervallo numerico definito come 601-800, 201-250, etc...

Per avere la reale posizione del world_rank si è deciso di utilizzare la funzione str.extract per estrarre il valore numerico e pulire la colonna 'world_rank'. Per quanto riguarda i valori ad intervallo si è andato a considerare il valore centrale fra i due estremi ed inserito nella nuova colonna 'world_rank_position'. Per tale motivo si ha la conversitione da tipo object a tipo float.

Si utilizza la funzione numpy.where in cui viene messa come primo elemento la condizione, come secondo cosa fare se la condizione in questione è soddisfatta e come terzo se non è soddisfatta.

Tale pulizia viene effettuata su tutti e tre i database forniti.



### Pulizia timesDb

In [28]:
timesDb_most_recent['world_rank_min'] = timesDb_most_recent['world_rank'].str.extract('=*(\d+)-*').astype(float)
timesDb_most_recent['world_rank_max'] = timesDb_most_recent['world_rank'].str.extract('-(\d+)').astype(float)

In [29]:
timesDb_most_recent['world_rank_max'] = np.where((timesDb_most_recent['world_rank_max'] >= 0), 
                                                 timesDb_most_recent['world_rank_max'], 
                                                 timesDb_most_recent['world_rank_min'])
timesDb_most_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_min,world_rank_max
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,601.0,800.0
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,201.0,250.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,251.0,300.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,106.0,106.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,301.0,350.0


In [30]:
timesDb_most_recent['world_rank_position'] = (timesDb_most_recent['world_rank_max'] + timesDb_most_recent['world_rank_min']) / 2
timesDb_most_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_min,world_rank_max,world_rank_position
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,601.0,800.0,700.5
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,201.0,250.0,225.5
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,251.0,300.0,275.5
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,106.0,106.0,106.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,301.0,350.0,325.5


### Pulizia shanghaiDb

In [31]:
shanghaiDb_most_recent = shanghaiDb.loc[shanghaiDb.groupby('university_name')['year'].idxmax()]
shanghaiDb_most_recent.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
4697,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015
4797,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015
4497,101-150,Aix Marseille University,5-6,NaN,13.6,0.0,15.2,20.3,50.1,27.6,2015
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011


In [32]:
shanghaiDb_most_recent['world_rank_min'] = shanghaiDb_most_recent['world_rank'].str.extract('=*(\d+)-*').astype(float)
shanghaiDb_most_recent['world_rank_max'] = shanghaiDb_most_recent['world_rank'].str.extract('-(\d+)').astype(float)

In [33]:
shanghaiDb_most_recent['world_rank_max'] = np.where((shanghaiDb_most_recent['world_rank_max'] >= 0), 
                                                    shanghaiDb_most_recent['world_rank_max'], 
                                                    shanghaiDb_most_recent['world_rank_min'])
shanghaiDb_most_recent.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_min,world_rank_max
4697,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015,301.0,400.0
4797,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015,401.0,500.0
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,73.0,73.0
4497,101-150,Aix Marseille University,5-6,NaN,13.6,0.0,15.2,20.3,50.1,27.6,2015,101.0,150.0
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011,102.0,150.0


In [34]:
shanghaiDb_most_recent['world_rank_position'] = (shanghaiDb_most_recent['world_rank_max'] + 
                                                 shanghaiDb_most_recent['world_rank_min']) / 2
shanghaiDb_most_recent.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_min,world_rank_max,world_rank_position
4697,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015,301.0,400.0,350.5
4797,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015,401.0,500.0,450.5
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,73.0,73.0,73.0
4497,101-150,Aix Marseille University,5-6,NaN,13.6,0.0,15.2,20.3,50.1,27.6,2015,101.0,150.0,125.5
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011,102.0,150.0,126.0


### Pulizia cwurDb

In [35]:
cwurDb_most_recent = cwurDb.loc[cwurDb.groupby('institution')['year'].idxmax()]
cwurDb_most_recent.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1981,782,AGH University of Science and Technology,Poland,4,267,279,218,708,891,511,896.0,398,44.26,2015
1764,565,Aalborg University,Denmark,5,367,439,218,596,660,287,520.0,401,44.59,2015
1620,421,Aalto University,Finland,4,367,224,218,336,403,645,424.0,246,45.10,2015
1321,122,Aarhus University,Denmark,2,310,165,106,91,81,101,95.0,154,49.65,2015
2013,814,Aberystwyth University,United Kingdom,59,367,567,218,842,552,511,754.0,607,44.22,2015


### Normalizzazione del nome delle università

Analizzando i diversi database si è notato che i nomi delle università contenuti nella colonna 'university_name' possono assumere valori differenti nonostante puntino alla stessa università. Si è deciso, allora, di normalizzare tali valori, grazie all'uso del database school_and_country__table, in modo che si avesse un nome comune per tutti i database relativo alle differenti università. 

La nuova colonna prenderà il nome di 'university_name_normalized'.

In [36]:
school_and_country_table = pd.read_csv('dataset_progetto/school_and_country_table.csv')
school_and_country_table

,school_name,country
0,Harvard University,United States of America
1,California Institute of Technology,United States of America
2,Massachusetts Institute of Technology,United States of America
3,Stanford University,United States of America
4,Princeton University,United States of America
...,...,...
813,Xidian University,China
814,Yeungnam University,South Korea
815,Yıldız Technical University,Turkey
816,Yokohama City University,Japan


In [37]:
def normalize_function (column_value):
    
    for university in school_and_country_table['school_name']:
        if column_value == university:
            return column_value
        
    pattern = re.compile(column_value)
    for university in school_and_country_table['school_name']:
        occurrence = pattern.search(university)
        if occurrence:
            return university
        else:
            university_pattern = re.compile(university)
            match_university = university_pattern.search(column_value)
            if match_university:
                return university

Applico la funzione a tutti e tre i database.

In [38]:
shanghaiDb_most_recent['university_name_normalized'] = shanghaiDb_most_recent['university_name'].apply(normalize_function)

In [39]:
timesDb_most_recent['university_name_normalized'] = timesDb_most_recent['university_name'].apply(normalize_function)

In [40]:
cwurDb_most_recent['university_name_normalized'] = cwurDb_most_recent['institution'].apply(normalize_function)

Dalla normalizzazione della colonna 'university_name' sono emersi dei valori None poiché la funzione normalize_function non ha trovato nessun match fra i nomi delle università nel database shanghaiDb con i nomi contenuti nella colonna 'school_name' del database school_and_country_table. Si decide di rimuovere tali valori.

In [41]:
shanghaiDb_most_recent[['university_name', 'university_name_normalized']].tail()

,university_name,university_name_normalized
3512,Yamaguchi University,None
4695,Yeshiva University,Yeshiva University
4696,Yonsei University,Yonsei University
4396,York University,York University
4546,Zhejiang University,Zhejiang University


In [42]:
shanghaiDb_most_recent.drop(shanghaiDb_most_recent[shanghaiDb_most_recent['university_name_normalized'].isnull()].index, inplace = True)

In [43]:
timesDb_most_recent.drop(timesDb_most_recent[timesDb_most_recent['university_name_normalized'].isnull()].index, inplace = True)

In [44]:
cwurDb_most_recent.drop(cwurDb_most_recent[cwurDb_most_recent['university_name_normalized'].isnull()].index, inplace = True)

Dopo aver normalizzato il campo 'university_name' e rimosso i valori None vado a ricalcolare le università con i dati più recenti.

In [45]:
shanghaiDb_most_recent_norm = shanghaiDb_most_recent.loc[shanghaiDb_most_recent.groupby('university_name_normalized')['year'].idxmax()]
shanghaiDb_most_recent_norm

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_min,world_rank_max,world_rank_position,university_name_normalized
4697,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015,301.0,400.0,350.5,Aalborg University
4797,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015,401.0,500.0,450.5,Aalto University
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,73.0,73.0,73.0,Aarhus University
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011,102.0,150.0,126.0,Aix-Marseille University
4798,401-500,Aristotle University of Thessaloniki,2,NaN,0.0,0.0,4.9,1.5,34.0,16.8,2015,401.0,500.0,450.5,Aristotle University of Thessaloniki
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407,11,Yale University,9,54.5,47.6,50.4,51.0,58.8,63.0,37.8,2015,11.0,11.0,11.0,Yale University
4695,201-300,Yeshiva University,79-102,NaN,0.0,0.0,6.3,21.7,31.5,21.6,2015,201.0,300.0,250.5,Yeshiva University
4696,201-300,Yonsei University,2-5,NaN,0.0,0.0,8.6,4.9,52.1,21.4,2015,201.0,300.0,250.5,Yonsei University
4396,401-500,York University,19-21,NaN,0.0,0.0,0.0,10.1,30.6,12.4,2014,401.0,500.0,450.5,York University


In [46]:
timesDb_most_recent_norm = timesDb_most_recent.loc[timesDb_most_recent.groupby('university_name_normalized')['year'].idxmax()]
timesDb_most_recent_norm.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_min,world_rank_max,world_rank_position,university_name_normalized
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,601.0,800.0,700.5,AGH University of Science and Technology
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,201.0,250.0,225.5,Aalborg University
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,251.0,300.0,275.5,Aalto University
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,106.0,106.0,106.0,Aarhus University
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,301.0,350.0,325.5,Aberystwyth University


In [47]:
cwurDb_most_recent_norm = cwurDb_most_recent.loc[cwurDb_most_recent.groupby('university_name_normalized')['year'].idxmax()]
cwurDb_most_recent_norm.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,university_name_normalized
1981,782,AGH University of Science and Technology,Poland,4,267,279,218,708,891,511,896.0,398,44.26,2015,AGH University of Science and Technology
1764,565,Aalborg University,Denmark,5,367,439,218,596,660,287,520.0,401,44.59,2015,Aalborg University
1620,421,Aalto University,Finland,4,367,224,218,336,403,645,424.0,246,45.10,2015,Aalto University
1321,122,Aarhus University,Denmark,2,310,165,106,91,81,101,95.0,154,49.65,2015,Aarhus University
2013,814,Aberystwyth University,United Kingdom,59,367,567,218,842,552,511,754.0,607,44.22,2015,Aberystwyth University


Vado a calcolare 3 nuove tabelle in cui sono presenti solamente 'university_name_normalized' e 'world_rank_position'.

In [48]:
shanghaiDb_most_recent_rank = shanghaiDb_most_recent_norm[['university_name_normalized', 'world_rank_position']]
shanghaiDb_most_recent_rank

,university_name_normalized,world_rank_position
4697,Aalborg University,350.5
4797,Aalto University,450.5
4469,Aarhus University,73.0
3115,Aix-Marseille University,126.0
4798,Aristotle University of Thessaloniki,450.5
...,...,...
4407,Yale University,11.0
4695,Yeshiva University,250.5
4696,Yonsei University,250.5
4396,York University,450.5


In [49]:
timesDb_most_recent_rank = timesDb_most_recent_norm[['university_name_normalized', 'world_rank_position']]
timesDb_most_recent_rank.head()

,university_name_normalized,world_rank_position
2405,AGH University of Science and Technology,700.5
2003,Aalborg University,225.5
2056,Aalto University,275.5
1908,Aarhus University,106.0
2105,Aberystwyth University,325.5


In [50]:
cwurDb_most_recent_rank = cwurDb_most_recent_norm[['university_name_normalized', 'world_rank']]
cwurDb_most_recent_rank.head()

,university_name_normalized,world_rank
1981,AGH University of Science and Technology,782
1764,Aalborg University,565
1620,Aalto University,421
1321,Aarhus University,122
2013,Aberystwyth University,814


Unisco i tre database in modo da porter calcolare la differenza fra la posizione di massimo e minimo rank assunta dalla univerisità.

In [51]:
db_rank = pd.merge(timesDb_most_recent_rank, shanghaiDb_most_recent_rank, on = ['university_name_normalized'], 
                   suffixes = ('_times', '_shanghai'))
db_rank.head()

,university_name_normalized,world_rank_position_times,world_rank_position_shanghai
0,Aalborg University,225.5,350.5
1,Aalto University,275.5,450.5
2,Aarhus University,106.0,73.0
3,Aix-Marseille University,275.5,126.0
4,Aristotle University of Thessaloniki,700.5,450.5


In [52]:
db_rank1 = pd.merge(db_rank, cwurDb_most_recent_rank, on = ['university_name_normalized'], suffixes = ('_times', '_cwurd'))
db_rank1.head()

,university_name_normalized,world_rank_position_times,world_rank_position_shanghai,world_rank
0,Aalborg University,225.5,350.5,565
1,Aalto University,275.5,450.5,421
2,Aarhus University,106.0,73.0,122
3,Aix-Marseille University,275.5,126.0,206
4,Aristotle University of Thessaloniki,700.5,450.5,459


In [53]:
db_rank1.rename(columns = {'world_rank': 'world_rank_cwurd'}, inplace=True)
db_rank1

,university_name_normalized,world_rank_position_times,world_rank_position_shanghai,world_rank_cwurd
0,Aalborg University,225.5,350.5,565
1,Aalto University,275.5,450.5,421
2,Aarhus University,106.0,73.0,122
3,Aix-Marseille University,275.5,126.0,206
4,Aristotle University of Thessaloniki,700.5,450.5,459
...,...,...,...,...
427,Yale University,12.0,11.0,11
428,Yeshiva University,164.0,250.5,171
429,Yonsei University,325.5,250.5,98
430,York University,325.5,450.5,337


Controllo se si hanno valore nulli nei campi 'world_rank_position_' per le 3 colonne.

In [54]:
db_rank1['world_rank_position_times'].isnull().any()

False

In [55]:
db_rank1['world_rank_position_shanghai'].isnull().any()

False

In [56]:
db_rank1['world_rank_cwurd'].isnull().any()

False

Calcolo la differenza fra il valore massimo e il valore minimo utilizzando la funzione max(axis=1) in cui specifico che deve ricercare il valore fra le colonne (axis{index (0), columns (1)}).

In [57]:
db_rank1['rank_difference'] = db_rank1[['world_rank_position_times', 'world_rank_position_shanghai', 'world_rank_cwurd']].max(axis=1) \
                                - db_rank1[['world_rank_position_times', 'world_rank_position_shanghai', 'world_rank_cwurd']].min(axis=1)
db_rank1

,university_name_normalized,world_rank_position_times,world_rank_position_shanghai,world_rank_cwurd,rank_difference
0,Aalborg University,225.5,350.5,565,339.5
1,Aalto University,275.5,450.5,421,175.0
2,Aarhus University,106.0,73.0,122,49.0
3,Aix-Marseille University,275.5,126.0,206,149.5
4,Aristotle University of Thessaloniki,700.5,450.5,459,250.0
...,...,...,...,...,...
427,Yale University,12.0,11.0,11,1.0
428,Yeshiva University,164.0,250.5,171,86.5
429,Yonsei University,325.5,250.5,98,227.5
430,York University,325.5,450.5,337,125.0


# 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [58]:
timesDb_most_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,world_rank_min,world_rank_max,world_rank_position,university_name_normalized
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,601.0,800.0,700.5,AGH University of Science and Technology
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,201.0,250.0,225.5,Aalborg University
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,251.0,300.0,275.5,Aalto University
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,106.0,106.0,106.0,Aarhus University
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,301.0,350.0,325.5,Aberystwyth University


Si nota come i valori assunti dalla colonna 'num_students' sono di tipo object poiché è presente la ',' per indicare le migliaia americana. Si usa, quindi, str.replace per rimuoverla e convertire la colonna nel tipo float.

In [59]:
timesDb_most_recent['num_students'] = timesDb_most_recent['num_students'].str.replace(',','').astype(float)

In [60]:
timesDb_most_recent['num_students']

2405    35569.0
2003    17422.0
2056    16099.0
1908    23895.0
2105     9252.0
         ...   
1856     2400.0
2013     2218.0
1904     2429.0
1833     9666.0
2134     8788.0
Name: num_students, Length: 818, dtype: float64

Per calcolare il numero di ragazze e ragazzi esatto per ogni università utilizzo la colonna 'female_male_ratio' in cui sono presenti in valore percentuale. Per alcune università il valore è mancante e viene indicato con '-'.

In [61]:
timesDb_most_recent['perc_female'] = timesDb_most_recent['female_male_ratio'].str.extract('(\d+) :').astype(float)
timesDb_most_recent['perc_male'] = timesDb_most_recent['female_male_ratio'].str.extract(' : (\d+)').astype(float)

In [62]:
timesDb_most_recent['num_female'] = (timesDb_most_recent['num_students'] * timesDb_most_recent['perc_female']) / 100
timesDb_most_recent['num_male'] = (timesDb_most_recent['num_students'] * timesDb_most_recent['perc_male']) / 100
timesDb_most_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,...,female_male_ratio,year,world_rank_min,world_rank_max,world_rank_position,university_name_normalized,perc_female,perc_male,num_female,num_male
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,35569.0,...,-,2016,601.0,800.0,700.5,AGH University of Science and Technology,NaN,NaN,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,...,48 : 52,2016,201.0,250.0,225.5,Aalborg University,48.0,52.0,8362.56,9059.44
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,...,32 : 68,2016,251.0,300.0,275.5,Aalto University,32.0,68.0,5151.68,10947.32
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,...,54 : 46,2016,106.0,106.0,106.0,Aarhus University,54.0,46.0,12903.30,10991.70
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,...,48 : 52,2016,301.0,350.0,325.5,Aberystwyth University,48.0,52.0,4440.96,4811.04


Dal calcolo esatto del numero di ragazzi e ragazze escono valori con la virgola. Si effettua, quindi, un'approssimazione per difetto o eccesso in base al valore.

In [63]:
timesDb_most_recent['num_female'] = timesDb_most_recent[timesDb_most_recent['num_female'].notnull()]['num_female'].round()
timesDb_most_recent['num_male'] = timesDb_most_recent[timesDb_most_recent['num_male'].notnull()]['num_male'].round()

In [64]:
timesDb_most_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,...,female_male_ratio,year,world_rank_min,world_rank_max,world_rank_position,university_name_normalized,perc_female,perc_male,num_female,num_male
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,35569.0,...,-,2016,601.0,800.0,700.5,AGH University of Science and Technology,NaN,NaN,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,...,48 : 52,2016,201.0,250.0,225.5,Aalborg University,48.0,52.0,8363.0,9059.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,...,32 : 68,2016,251.0,300.0,275.5,Aalto University,32.0,68.0,5152.0,10947.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,...,54 : 46,2016,106.0,106.0,106.0,Aarhus University,54.0,46.0,12903.0,10992.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,...,48 : 52,2016,301.0,350.0,325.5,Aberystwyth University,48.0,52.0,4441.0,4811.0


In [65]:
country_num_female = timesDb_most_recent.groupby('country', as_index = False)['num_female'].sum()
country_num_male = timesDb_most_recent.groupby('country', as_index = False)['num_male'].sum()

In [66]:
country_students = pd.merge(country_num_female, country_num_male, on = 'country')
country_students

,country,num_female,num_male
0,Argentina,67191.0,41182.0
1,Australia,391736.0,321640.0
2,Austria,68364.0,66113.0
3,Bangladesh,21323.0,41393.0
4,Belarus,20219.0,9084.0
...,...,...,...
67,Unisted States of America,0.0,0.0
68,United Arab Emirates,9516.0,4931.0
69,United Kingdom,711814.0,613028.0
70,United States of America,1601453.0,1556324.0


Dall'analisi dei valori si può notare come ci siano degli errori di scrittura nel nome del paese.

In particolare: 
    
    Unted Kingdom con United Kingdom
    Unisted States of America con United States of America
    
Il valore del count effettuato su questi paesi risulta essere pari a 0. Si va ad indagare la presenza di errori nel nome di altri paesi andando a stampare tutti i paesi con valore pari a 0.

In [67]:
country_students[country_students['num_female'] == country_students['num_female'].min()]

,country,num_female,num_male
67,Unisted States of America,0.0,0.0
71,Unted Kingdom,0.0,0.0


In [68]:
country_students[country_students['num_male'] == country_students['num_male'].min()]

,country,num_female,num_male
67,Unisted States of America,0.0,0.0
71,Unted Kingdom,0.0,0.0


Si decide, quindi, di rimuovere le righe 67 e 71 dal database.

In [69]:
country_students.drop([67,71], inplace=True)

In [70]:
country_students

,country,num_female,num_male
0,Argentina,67191.0,41182.0
1,Australia,391736.0,321640.0
2,Austria,68364.0,66113.0
3,Bangladesh,21323.0,41393.0
4,Belarus,20219.0,9084.0
...,...,...,...
65,Uganda,18670.0,18670.0
66,Ukraine,17846.0,19250.0
68,United Arab Emirates,9516.0,4931.0
69,United Kingdom,711814.0,613028.0


# 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities).

In [71]:
timesDb_most_recent['ratio'] = timesDb_most_recent['perc_female'] / timesDb_most_recent['perc_male']
timesDb_most_recent['ratio']

2405         NaN
2003    0.923077
2056    0.470588
1908    1.173913
2105    0.923077
          ...   
1856    0.851852
2013    0.960784
1904    0.219512
1833    0.369863
2134    1.564103
Name: ratio, Length: 818, dtype: float64

Dopo aver calcolato il rapporto fra la percentuale di femmine sulla percentuale di uomini vado a verificare se sono presenti valori infiniti dovuti dalla presenza di un'università interamente composta da parte femminile (100/0).

In [72]:
timesDb_most_recent.loc[timesDb_most_recent['ratio'] == np.inf]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,...,year,world_rank_min,world_rank_max,world_rank_position,university_name_normalized,perc_female,perc_male,num_female,num_male,ratio
2227,401-500,Ewha Womans University,South Korea,23.9,34.3,18.3,50.4,57.6,-,17625.0,...,2016,401.0,500.0,450.5,Ewha Womans University,100.0,0.0,17625.0,0.0,inf


Sostituisco il valore infinito con 100 che rappresenta la totalità.

In [73]:
timesDb_most_recent['ratio'].replace({np.inf: 100}, inplace= True)

Calcolo il valor medio di tutti i rapporti ottenuti ai punti precedenti.

In [74]:
avg_ratio = timesDb_most_recent['ratio'].mean()
avg_ratio

1.2149571636836496

In [75]:
timesDb_most_recent.loc[timesDb_most_recent['ratio'] < avg_ratio][['university_name', 'ratio']]

,university_name,ratio
2003,Aalborg University,0.923077
2056,Aalto University,0.470588
1908,Aarhus University,1.173913
2105,Aberystwyth University,0.923077
2406,Ajou University,0.492537
...,...,...
2104,Zhejiang University,0.694915
1856,École Normale Supérieure,0.851852
2013,École Normale Supérieure de Lyon,0.960784
1904,École Polytechnique,0.219512


# 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

Salvo il risultato precedente in 'timesDb_most_recent_below_avg' in modo da porter eseguire un groupby per country e ottenere la somma di tutti gli studenti delle università.

In [76]:
timesDb_most_recent_below_avg = timesDb_most_recent.loc[timesDb_most_recent['ratio'] < avg_ratio]

In [77]:
timesDb_most_recent_below_avg_for_country = timesDb_most_recent_below_avg.groupby('country', as_index = False)['num_students'].sum()
timesDb_most_recent_below_avg_for_country.head()

,country,num_students
0,Australia,295021.0
1,Austria,79242.0
2,Bangladesh,62716.0
3,Belgium,116129.0
4,Brazil,438476.0


Dal dataframe timesDb_most_recent vado a raggrupare per country e ottengo il numero totale di studenti per ogni paese (questo sarà il denominatore nel calcolo del point).

In [78]:
timesDb_most_recent_for_country = timesDb_most_recent.groupby('country', as_index = False)['num_students'].sum()
timesDb_most_recent_for_country.head()

,country,num_students
0,Argentina,108373.0
1,Australia,743627.0
2,Austria,134477.0
3,Bangladesh,62716.0
4,Belarus,29303.0


In [79]:
results = pd.merge(timesDb_most_recent_below_avg_for_country, timesDb_most_recent_for_country, on = 'country', 
                   suffixes = ('', '_tot') )
results['results'] = results['num_students'] / results['num_students_tot']
results.head()

,country,num_students,num_students_tot,results
0,Australia,295021.0,743627.0,0.396733
1,Austria,79242.0,134477.0,0.589261
2,Bangladesh,62716.0,62716.0,1.000000
3,Belgium,116129.0,169661.0,0.684477
4,Brazil,438476.0,534688.0,0.820060


# 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name.

In [80]:
educ_att_sup = pd.read_csv('dataset_progetto/educational_attainment_supplementary_data.csv')
educ_att_sup.head(3)

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN


In [81]:
educ_att_sup[(educ_att_sup['country_name'].isnull()) | (educ_att_sup['series_name'].isnull())]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79054,Last Updated: 10/20/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Utilizzo la funzione dropna per rimuovere le righe che hanno come valori mancanti una dei valori delle colonne 'country_name' o 'series_name'.

Successivamente effettuo un ulteriore controllo.

In [82]:
educ_att_sup.dropna(subset = ['country_name', 'series_name'], inplace = True)

In [83]:
educ_att_sup[(educ_att_sup['country_name'].isnull()) | (educ_att_sup['series_name'].isnull())]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015


# 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value.

In [84]:
educ_att_sup.head(3)

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN


Utilizzo la funzione melt() per modificare la struttura del database. Assegno una nuova colonna chiamata 'year' a cui inserisco tutte le date che nella tabella 'educ_att_sup' erano delle colonne.

In [85]:
melt_educ_att_sup = pd.melt(educ_att_sup, id_vars = ['country_name', 'series_name'], 
                            value_vars = ['1985', '1986', '1987', '1990', '1991', '1992', '1993', '1995', '1996', '1997', '1998', 
                                          '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
                                          '2010', '2011', '2012', '2013', '2015'], var_name = 'year')
melt_educ_att_sup.head()

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54


# 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

Applico la funzione 'normalize_function' a 'university_name', che contiene il nome delle università, ai 3 database di partenza e vado a salvare tale valore in una nuova colonna, 'university_name_normalized'. Questo passaggio mi permette di andare a valutare il ranking dell'università per ogni anno tenendo conto di più classificazioni in anni differenti. 

In [86]:
shanghaiDb['university_name_normalized'] = shanghaiDb['university_name'].apply(normalize_function)

In [87]:
timesDb['university_name_normalized'] = timesDb['university_name'].apply(normalize_function)

In [88]:
cwurDb['university_name_normalized'] = cwurDb['institution'].apply(normalize_function)

Come nel punto 4. dalla normalizzazione della colonna contente i nomi delle università sono emersi dei valori None poiché la funzione normalize_function non ha trovato nessun match fra i nomi delle università nel database shanghaiDb con i nomi contenuti nella colonna 'school_name' del database school_and_country_table. Si decide di rimuovere tali valori.

In [89]:
shanghaiDb[['university_name', 'university_name_normalized']].tail()

,university_name,university_name_normalized
4892,University of Trieste,University of Trieste
4893,University of Zaragoza,University of Zaragoza
4894,Utah State University,None
4895,Vienna University of Technology,Vienna University of Technology
4896,Wake Forest University,Wake Forest University


Rimuovo le rows contenenti None in 'university_name_normalized' e creo delle nuove tabelle.

In [90]:
shanghaiDb_norm = shanghaiDb.drop(shanghaiDb[shanghaiDb['university_name_normalized'].isnull()].index)

In [91]:
timesDb_norm = timesDb.drop(timesDb[timesDb['university_name_normalized'].isnull()].index)

In [92]:
cwurDb_norm = cwurDb.drop(cwurDb[cwurDb['university_name_normalized'].isnull()].index)

Tramite la funzione set() mi creo un dizionario contenente le università per tutti i database. 

Tale funzione è definita come set(iterable), dove con iterable si richiede una un qualsiasi oggetto iterabile. Si passa quindi la serie ottenuta dai db.

In [93]:
shanghaiDb_norm_university = set(shanghaiDb_norm['university_name_normalized'])

In [94]:
timesDb_norm_university = set(timesDb_norm['university_name_normalized'])

In [95]:
cwurDb_norm_university = set(cwurDb_norm['university_name_normalized'])

Mi creo un dizionario chiamato 'appearence_of_university' su cui successivamente aggiungero e conterò il numero di volte che è contenuta l'università nei dizionari ottenuti alle righe precedenti.

In [96]:
appearence_of_university = {}

In [97]:
for university in shanghaiDb_norm_university:
    appearence_of_university[university] = 1
for university in timesDb_norm_university:
    if university not in appearence_of_university:
        appearence_of_university[university] = 1
    else:
        appearence_of_university[university] += 1
for university in cwurDb_norm_university:
    if university not in appearence_of_university:
        appearence_of_university[university] = 1
    else:
        appearence_of_university[university] += 1

In [98]:
appearence_of_university

{'Osaka Prefecture University': 3,
 'University of Toronto': 3,
 'Laval University': 3,
 'University of Aberdeen': 3,
 'Pennsylvania State University': 3,
 'University of Wyoming': 3,
 'Flinders University': 3,
 'University of Strasbourg': 3,
 'University of South Carolina': 3,
 'University of Oxford': 3,
 'Boston University': 3,
 'National Central University': 3,
 'University of Cambridge': 3,
 'São Paulo State University (UNESP)': 2,
 'Syracuse University': 3,
 'Bar-Ilan University': 3,
 'University of Zurich': 3,
 'ETH Zurich – Swiss Federal Institute of Technology Zurich': 2,
 'Memorial University of Newfoundland': 3,
 'University of Vienna': 3,
 'University of Leicester': 3,
 'University of Minnesota': 3,
 'University of British Columbia': 3,
 'University of Maryland, Baltimore County': 3,
 'Kanazawa University': 3,
 'National Autonomous University of Mexico': 3,
 'Tianjin University': 3,
 'Karlsruhe Institute of Technology': 3,
 'University College London': 3,
 'University of Hon

# 11. In the times ranking, compute the number of times each university appears.

In [99]:
timesDb_count = timesDb_norm.groupby('university_name').size()
timesDb_count

university_name
AGH University of Science and Technology    1
Aalborg University                          5
Aalto University                            5
Aarhus University                           6
Aberystwyth University                      5
                                           ..
École Normale Supérieure                    6
École Normale Supérieure de Lyon            6
École Polytechnique                         6
École Polytechnique Fédérale de Lausanne    6
Örebro University                           1
Length: 818, dtype: int64

# 12. Find the universities that appear at most twice in the times ranking.

In [100]:
timesDb_count.loc[timesDb_count <= 2]

university_name
AGH University of Science and Technology    1
Adam Mickiewicz University                  1
Aix-Marseille University                    1
Ajou University                             1
Alexandru Ioan Cuza University              1
                                           ..
Yokohama City University                    1
Yokohama National University                1
Yuan Ze University                          2
Yıldız Technical University                 1
Örebro University                           1
Length: 408, dtype: int64

# 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In modo analogo al punto 4, con la differenza che si lavorava su tutti i data presenti nei db, eseguo gli stessi passaggi.

### Pulizia timesDb

In [101]:
timesDb['world_rank_min'] = timesDb['world_rank'].str.extract('=*(\d+)-*').astype(float)
timesDb['world_rank_max'] = timesDb['world_rank'].str.extract('-(\d+)').astype(float)

In [102]:
timesDb['world_rank_max'] = np.where((timesDb['world_rank_max'] >= 0), 
                                      timesDb['world_rank_max'], 
                                      timesDb['world_rank_min'])
timesDb.head(3)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,university_name_normalized,world_rank_min,world_rank_max
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011,Harvard University,1.0,1.0
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011,California Institute of Technology,2.0,2.0
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011,Massachusetts Institute of Technology,3.0,3.0


In [103]:
timesDb_world_rank = timesDb[['university_name_normalized', 'year', 'world_rank', 'world_rank_min', 'world_rank_max']]
timesDb_world_rank

,university_name_normalized,year,world_rank,world_rank_min,world_rank_max
0,Harvard University,2011,1,1.0,1.0
1,California Institute of Technology,2011,2,2.0,2.0
2,Massachusetts Institute of Technology,2011,3,3.0,3.0
3,Stanford University,2011,4,4.0,4.0
4,Princeton University,2011,5,5.0,5.0
...,...,...,...,...,...
2598,Yeungnam University,2016,601-800,601.0,800.0
2599,Yıldız Technical University,2016,601-800,601.0,800.0
2600,Yokohama City University,2016,601-800,601.0,800.0
2601,Yokohama National University,2016,601-800,601.0,800.0


### Pulizia shanghaiDb

In [104]:
shanghaiDb['world_rank_min'] = shanghaiDb['world_rank'].str.extract('=*(\d+)-*').astype(float)
shanghaiDb['world_rank_max'] = shanghaiDb['world_rank'].str.extract('-(\d+)').astype(float)

In [105]:
shanghaiDb['world_rank_max'] = np.where((shanghaiDb['world_rank_max'] >= 0), 
                                         shanghaiDb['world_rank_max'], 
                                         shanghaiDb['world_rank_min'])
shanghaiDb.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,university_name_normalized,world_rank_min,world_rank_max
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,Harvard University,1.0,1.0
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,University of Cambridge,2.0,2.0
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,Stanford University,3.0,3.0
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,"University of California, Berkeley",4.0,4.0
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,Massachusetts Institute of Technology,5.0,5.0


In [106]:
shanghaiDb_world_rank = shanghaiDb[['university_name_normalized', 'year', 'world_rank', 'world_rank_min', 'world_rank_max']]
shanghaiDb_world_rank

,university_name_normalized,year,world_rank,world_rank_min,world_rank_max
0,Harvard University,2005,1,1.0,1.0
1,University of Cambridge,2005,2,2.0,2.0
2,Stanford University,2005,3,3.0,3.0
3,"University of California, Berkeley",2005,4,4.0,4.0
4,Massachusetts Institute of Technology,2005,5,5.0,5.0
...,...,...,...,...,...
4892,University of Trieste,2015,401-500,401.0,500.0
4893,University of Zaragoza,2015,401-500,401.0,500.0
4894,None,2015,401-500,401.0,500.0
4895,Vienna University of Technology,2015,401-500,401.0,500.0


### Pulizia cwurDb

In [107]:
cwurDb['world_rank'] = cwurDb['world_rank'].astype(float)

In [108]:
cwurDb_world_rank =  cwurDb[['university_name_normalized', 'year', 'world_rank']]

Per eseguire le join fra le tabelle utilizzo la colonna 'univeristy_name_normalized' inserita al point 10 tramite la funzione 'normalize_function'. Il risultato di questa normalizzazione aveva provocato dei valori None che si era deciso di rimuovere.

In [109]:
db_rank_position = pd.merge(timesDb_world_rank, shanghaiDb_world_rank, on = ('university_name_normalized', 'year'), 
                            suffixes = ('_times', '_shanghai'))
db_rank_position.head()

,university_name_normalized,year,world_rank_times,world_rank_min_times,world_rank_max_times,world_rank_shanghai,world_rank_min_shanghai,world_rank_max_shanghai
0,Harvard University,2011,1,1.0,1.0,1,1.0,1.0
1,California Institute of Technology,2011,2,2.0,2.0,6,6.0,6.0
2,Massachusetts Institute of Technology,2011,3,3.0,3.0,3,3.0,3.0
3,Stanford University,2011,4,4.0,4.0,2,2.0,2.0
4,Princeton University,2011,5,5.0,5.0,7,7.0,7.0


In [110]:
db_rank_position1 = pd.merge(db_rank_position, cwurDb_world_rank, on = ('university_name_normalized', 'year'), 
                             suffixes = ('', '_cwurd'))
db_rank_position1.head()

,university_name_normalized,year,world_rank_times,world_rank_min_times,world_rank_max_times,world_rank_shanghai,world_rank_min_shanghai,world_rank_max_shanghai,world_rank
0,California Institute of Technology,2012,1,1.0,1.0,6,6.0,6.0,5.0
1,Harvard University,2012,2,2.0,2.0,1,1.0,1.0,1.0
2,Stanford University,2012,2,2.0,2.0,2,2.0,2.0,3.0
3,University of Oxford,2012,4,4.0,4.0,10,10.0,10.0,7.0
4,Princeton University,2012,5,5.0,5.0,7,7.0,7.0,6.0


In [111]:
db_rank_position1.rename(columns={'world_rank': 'world_rank_cwurd'}, inplace=True)
db_rank_position1.head(3)

,university_name_normalized,year,world_rank_times,world_rank_min_times,world_rank_max_times,world_rank_shanghai,world_rank_min_shanghai,world_rank_max_shanghai,world_rank_cwurd
0,California Institute of Technology,2012,1,1.0,1.0,6,6.0,6.0,5.0
1,Harvard University,2012,2,2.0,2.0,1,1.0,1.0,1.0
2,Stanford University,2012,2,2.0,2.0,2,2.0,2.0,3.0


In [112]:
result = db_rank_position1.loc[(((db_rank_position1['world_rank_cwurd'] >= db_rank_position1['world_rank_min_times']) 
                                    & (db_rank_position1['world_rank_cwurd'] <= db_rank_position1['world_rank_max_times'])) 
                                   & (((db_rank_position1['world_rank_cwurd'] >= db_rank_position1['world_rank_min_shanghai']) 
                                & (db_rank_position1['world_rank_cwurd'] <= db_rank_position1['world_rank_max_shanghai']))))]

In [113]:
result[['university_name_normalized','year', 'world_rank_times', 'world_rank_shanghai', 'world_rank_cwurd']]

,university_name_normalized,year,world_rank_times,world_rank_shanghai,world_rank_cwurd
78,Stanford University,2013,2,2,2.0
273,University of Southampton,2014,146,101-150,146.0
324,Sungkyunkwan University (SKKU),2014,201-225,201-300,204.0
413,University of Vermont,2014,301-350,301-400,320.0
429,Temple University,2014,351-400,301-400,390.0
630,Newcastle University,2015,201-225,201-300,212.0
644,Autonomous University of Barcelona,2015,226-250,201-300,241.0
648,Laval University,2015,226-250,201-300,229.0
660,Medical University of Vienna,2015,251-275,201-300,265.0
675,University of Waterloo,2015,251-275,201-300,269.0
